In [ ]:
!pip install groq

In [ ]:
from dotenv import load_dotenv
import os
from groq import Groq
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from IPython.display import Markdown, display, Audio

In [ ]:
load_dotenv(override=True)
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY', 'your-key-if-not-using-env')
groq = Groq(api_key=os.environ["GROQ_API_KEY"])

In [ ]:
db_name = "guru_db"
LLM_MODEL_NAME = "llama-3.1-8b-instant"#"openai/gpt-oss-20b",#"groq/compound"
TTS_MODEL_NAME = "playai-tts"
TTS_VOICE = "Atlas-PlayAI"
TTS_RESPONSE_FORMAT = "wav"
embeddings = OpenAIEmbeddings()

In [ ]:
vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
# Test
docs = retriever.get_relevant_documents("What are your Certifications?")
print(docs)

In [ ]:
# Function to convert the docs to string
def retDocToStr(docs):
    doc_str = ""
    for doc in docs:
        doc_str += f"{doc.metadata['doc_type']}" + f"\n{doc.page_content}\n\n"
    return doc_str


In [ ]:
name = "Guru Deep Singh"
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
You are allowed to provide all information given to you as Context including URLs, email,etc. \
ALWAYS answer as {name}. Never say your are a Large Language Model. Use ONLY the following context to answer. If the answer isn't in the context, say \"I don't know."


In [ ]:
# Function to to get LLM response
def ask_llm(prompt, vecStore, k=10):
    retriever = vecStore.as_retriever(search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(prompt)
    context = retDocToStr(docs)
    msgs = [{"role":"system","content":system_prompt},
            {"role":"user","content":f"### CONTEXT: {context}\n\nUSER QUESTION: {prompt}"}]
    print("RAG CONTEXT: ", msgs)
    resp = groq.chat.completions.create(
        model=LLM_MODEL_NAME,
        messages=msgs,
        temperature=0.5,
        stream=False,
    )
    return resp.choices[0].message.content

In [ ]:
# Test
prompt = "Tell me about yourself."
out = ask_llm(prompt, vectorstore, 20)
 display(Markdown(out))

In [ ]:
# Function to run TTS
def textToSpeech(text):
    response = groq.audio.speech.create(
        model=TTS_MODEL_NAME,
        voice=TTS_VOICE,
        input=text,
        response_format=TTS_RESPONSE_FORMAT
    )
    return response

In [ ]:
# Function to run LLM + TTS
def ask_llm_speech(prompt, vecStore, k=10, speech=True):
    response = ask_llm(prompt, vecStore, k)
    if speech:
        return response, textToSpeech(response)
    else:
        return response
    

In [ ]:
prompt = "What are your GRE and IELTS score?"
res = ask_llm_speech(prompt, vectorstore, 20, True)

if isinstance(res, tuple):            # two outputs: (text, audio)
    answer, voice = res
else:                                  # one output: text only
    answer, voice = res, None
    

In [ ]:
#Answer
display(Markdown(answer))

if voice:
    # Voice
    speech_file_path = "speech.wav"
    voice.write_to_file(speech_file_path)  # helper on BinaryAPIResponse
    display(Audio(speech_file_path))